# 第02章 — 力

- ニュートンの第二法則: 加速度 = 合力 / 質量。これを積分して速度・位置を更新する。
- 代表的な力: 重力、抗力、摩擦、ばね力（フックの法則）。
- 力は毎フレームベクトルとして蓄積し、積分後にリセットする。
- 力の強さは距離でスケールすることが多い（重力なら距離の二乗に反比例）。

試してみよう: 抗力係数や初期距離を変えて、運動の安定/不安定さを試す。

In [ ]:
import mathclass Vector:    def __init__(self, x=0.0, y=0.0):        self.x, self.y = float(x), float(y)    def __add__(self, other):        return Vector(self.x + other.x, self.y + other.y)    def __sub__(self, other):        return Vector(self.x - other.x, self.y - other.y)    def __mul__(self, k):        return Vector(self.x * k, self.y * k)    __rmul__ = __mul__    def mag(self):        return math.hypot(self.x, self.y)    def normalize(self):        m = self.mag()        if m == 0:            return Vector()        return self * (1.0 / m)    def limit(self, max_mag):        m = self.mag()        if m > max_mag:            return self.normalize() * max_mag        return self    def __repr__(self):        return f"Vector({self.x:.2f}, {self.y:.2f})"class Body:    def __init__(self, pos, mass=1.0):        self.pos = pos        self.vel = Vector(0, 0)        self.acc = Vector(0, 0)        self.mass = mass    def apply_force(self, force):        # F = m * a なので a = F / m        self.acc += force * (1 / self.mass)    def update(self, dt=1.0):        self.vel += self.acc * dt        self.pos += self.vel * dt        self.acc = Vector(0, 0)def gravity(attractor, mover, G=1.0):    dir_vec = attractor.pos - mover.pos    dist = max(0.5, dir_vec.mag())    strength = G * attractor.mass * mover.mass / (dist * dist)    return dir_vec.normalize() * strengthattractor = Body(Vector(0, 0), mass=20)mover = Body(Vector(5, 0), mass=2)for step in range(5):    g = gravity(attractor, mover, G=0.8)    drag = mover.vel * -0.05    mover.apply_force(g)    mover.apply_force(drag)    mover.update(0.5)    print(f"step {step}: pos={mover.pos} vel={mover.vel}")

### 追加例: 抗力付きの落下（終端速度の例）

In [ ]:
import math

class Body:
    def __init__(self, mass=1.0, drag_coef=0.2):
        self.v = 0.0
        self.y = 50.0
        self.mass = mass
        self.drag = drag_coef

    def step(self, dt=0.1, g=-9.8):
        # 抗力: Fd = -c * v * |v|
        drag = -self.drag * self.v * abs(self.v)
        acc = (g * self.mass + drag) / self.mass
        self.v += acc * dt
        self.y += self.v * dt
        return self.y, self.v

body = Body(mass=1.0, drag_coef=0.35)
for i in range(12):
    y, v = body.step()
    print(f"step {i:02d}: y={y:6.2f} v={v:6.2f}")